In [2]:
import urllib.request
from datetime import datetime
import camelot
import pandas as pd
%matplotlib qt

In [7]:
EVT_LOG_URI = 'https://uwpd.wisc.edu/content/uploads/2020/10/Building-Manager-Log.pdf'

In [8]:
now = datetime.now()
fname = f'eventlog-{now.year}-{now.month}-{now.day}.pdf'
urllib.request.urlretrieve(EVT_LOG_URI, fname)

('eventlog-2021-9-29.pdf', <http.client.HTTPMessage at 0x7f38ce6497c0>)

In [37]:
EVT_LOG_AREAS = {
    'first': ['0,555,800,25'],
    'rest': ['0,600,800,25'],
}
tables = camelot.read_pdf(fname, flavor='stream', pages='2-end', table_areas=EVT_LOG_AREAS['rest'])

In [133]:
tables[0].df.iloc[0,3]

'Location\nAddress'

In [143]:
# clean up a page's table
def postprocess(df):
    df = df.copy()
    
    needs_loc_split = df.iloc[0,3] == 'Location\nAddress'
    df.drop(0, inplace=True)
    if needs_loc_split:
        df.rename(columns={0: 'Event #', 1: 'Date & Time', 2: 'Nature', 3: 'Location & Address', 4: 'Status'}, inplace=True)
        df[['Location', 'Address']] = df['Location & Address'].str.split('\n', 1, expand=True)
        df.drop(columns='Location & Address', inplace=True)
    else:
        df.rename(columns={0: 'Event #', 1: 'Date & Time', 2: 'Nature', 3: 'Location', 4: 'Address', 5: 'Status'}, inplace=True)
    
    # fixup last page stuff
    if '/' in df.iloc[-1,0]: # date in evt column
        df.drop(, rows=-1, inplace=True)

    df['Event #'] = pd.to_numeric(df['Event #'])
    # df[1] = df[1].apply(lambda dts: datetime.strptime(dts, '%m/%d/%y %H:%M %p'))
    df['Date & Time'] = pd.to_datetime(df['Date & Time'])
    return df

In [4]:
CLERY_LOG_URI = 'https://uwpd.wisc.edu/content/uploads/2020/10/Clery-Crime-Fire-Log.pdf'
now = datetime.now()
fname = f'clery-{now.year}-{now.month}-{now.day}.pdf'
urllib.request.urlretrieve(CLERY_LOG_URI, fname)

('clery-2021-9-29.pdf', <http.client.HTTPMessage at 0x7fb26ee872b0>)

In [45]:
tables = camelot.read_pdf(fname, flavor='stream', pages='all',
                          table_areas=['0,530,760,0'],
                          columns=['100,180,280,330,465,637'],
                          split_text=True)

In [65]:
any(['END OF REPORT' in x for x in tables[-1].df.iloc[-1]])

True

In [42]:
camelot.plot(tables[0], kind='text').show()